# Relational Databse Implementaiton with SQLite3

### Importing Libraries needed for the applicaiton

In [59]:
import sqlalchemy
import datetime
import decimal
from sqlalchemy import exc
#from datetime import datetime
import sqlite3

# Imports the declarative_base object, which connects the database engine to the SQLAlchemy functionality of the models.
from sqlalchemy.ext.declarative import declarative_base

# Imports the Column, Integer, String classes from SQLAlchemy, which are used to help define the model attributes.
from sqlalchemy import Column, Integer, String, Float, BigInteger, DateTime, UniqueConstraint, DECIMAL

# Imports all the cinstraints, functionalities sqlalchemy offer
from sqlalchemy import *

# Imports the relationship() object, which are used to create the relationships between objects.
from sqlalchemy.orm import relationship

# Imports create_engine whihc is subsequently used for SQL operation
from sqlalchemy import create_engine


### Creating Relational Database using sqlalchemy and sqlite

In [61]:
# The first parameter is the name of the database.
# The second parameter (echo) is set to true; this will generate the activity log
DatabaseName = 'OnlineFoodDeliveryAppDemoRun01.db'
pathForSQLConn = 'D:\\Study\\Submissions\\Term 2\\Database\\Group Project\\'+DatabaseName
pathforEngine = 'sqlite:///'+DatabaseName
engine = create_engine(pathforEngine, echo = True)

print("Welcome to our OnlineFoodDelivery Platform")

Welcome to our OnlineFoodDelivery Platform


In [4]:

# Creates the Base class, which is what all models inherit from and how they get SQLAlchemy ORM functionality.
# Once base class is declared, any number of mapped classes can be defined in terms of it (a reference is passed to the class).
Base=declarative_base()

# Defines the Student class model to the customers database table.
# In ORM, the configurational process starts by describing the database tables and then by defining classes which will be mapped to those tables. 
# In SQLAlchemy, these two tasks are performed together. 
# A class in Declarative must have a __tablename__ attribute, and at least one Column which is part of a primary key. 

class Shop(Base):
    __tablename__ = 'shops'
    id = Column(Integer,primary_key=True)
    shop_name = Column(String(40),nullable=False,unique=True)
    shop_cuisine = Column(String(40),nullable=False)
    shop_addressLine1 = Column(String(20),nullable=False)
    shop_postalCode = Column(String(6),nullable=False)
    shop_contact = Column(Integer,nullable=False)
    items = relationship('Item',secondary='link')

class Item(Base):
    __tablename__ = 'items'
    id = Column(Integer,primary_key=True)
    item_name = Column(String(40),nullable=False,unique=True)
    item_price = Column(DECIMAL(7,3),nullable=False)
    shops = relationship('Shop',secondary='link')
    
    
# Define a Link class. It is linked to link table and contains shop_id and iyem_id attributes respectively referencing to primary keys of shops and items table.
class Link(Base):
    __tablename__ = 'link'
    shop_id = Column(Integer, ForeignKey('shops.id'), primary_key = True)
    item_id = Column(Integer, ForeignKey('items.id'), primary_key = True)


class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer,primary_key=True)
    cust_name = Column(String(30),nullable=False)
    cust_email = Column(String(40),nullable=False,unique=True)
    cust_addressLine1 = Column(String(40),nullable=False)
    cust_postalCode = Column(String(6),nullable=False)
    cust_contact = Column(Integer,nullable=False)
    placeOrders = relationship('Order',back_populates='custO',cascade = "all, delete, delete-orphan")

class Order(Base):
    __tablename__ = 'orders'
    id = Column(Integer,primary_key=True)
    cust_id = Column(Integer,ForeignKey('customers.id',ondelete="CASCADE",onupdate="CASCADE"),nullable=False)
    item_id = Column(Integer,ForeignKey('items.id'),primary_key=True)
    item_qty = Column(Integer,nullable=False)
    shop_id = Column(Integer,ForeignKey('shops.id'),primary_key=True)
    custO = relationship('Customer',back_populates='placeOrders')


class Coupon(Base):
    __tablename__ = 'coupons'
    coupon_code = Column(String(10),primary_key=True)
    coupon_discount = Column(Integer,nullable=False)
    coupon_status = Column(String,DefaultClause("ACTIVE"),nullable=False)
    coupon_validFrom = Column(Date,nullable=False)
    coupon_validTill = Column(Date,nullable=False)

class Invoice(Base):
    __tablename__ = 'invoice'
    invoice_id = Column(Integer,primary_key=True)
    order_id = Column(Integer,ForeignKey('orders.id',ondelete="CASCADE",onupdate="CASCADE"),unique=True,nullable=False)
    total_amt = Column(DECIMAL(7,3),nullable=False)
    discouted_price = Column(DECIMAL(7,3),nullable=False)
    coupon_code = Column(String(10),ForeignKey('coupons.coupon_code'))
    invoice_date = Column(Date,nullable=False)
    
    
class Feedback(Base):
    __tablename__ = 'feedback'
    feedback_id = Column(Integer,primary_key=True)
    order_id = Column(Integer,ForeignKey('orders.id',ondelete="CASCADE",onupdate="CASCADE"),nullable=False)
    feedback_text = Column(String(100),nullable=False)


# Each Table object is a member of larger collection known as MetaData and this object is available using the .metadata attribute of declarative base class. 
# The MetaData.create_all() method is, passing in our Engine as a source of database connectivity. For all tables that haven’t been created yet, it issues CREATE TABLE statements to the database.
# Create all tables in the engine. This is equivalent to "Create Table"
Base.metadata.create_all(engine)

2022-03-30 17:43:49,106 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 17:43:49,107 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("shops")
2022-03-30 17:43:49,130 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-30 17:43:49,131 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("shops")
2022-03-30 17:43:49,131 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-30 17:43:49,132 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("items")
2022-03-30 17:43:49,133 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-30 17:43:49,133 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("items")
2022-03-30 17:43:49,134 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-30 17:43:49,135 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("link")
2022-03-30 17:43:49,135 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-30 17:43:49,136 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("link")
2022-03-30 17:43:49,136 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-30 17

### Creating session object for lchemy and conn object for sqlite which can be used for querying the database as per needed

In [62]:
# The Session object enables us to interest with the database (it is a handle to the database)
from sqlalchemy.orm import sessionmaker

# Session class is defined using sessionmaker(), which is bound to the engine object created earlier
Session = sessionmaker(bind = engine)


### Inserting records into Shop and related items using many to many relashioship and retreiving it by joining multiple tables
#### This is many to many relationship (a shop can have multiple items and an item can be present in multiple shops)

In [63]:
#creating cursor for sqlite conneciton and session for alchemy
conn1=sqlite3.connect(pathForSQLConn)
cur=conn1.cursor()
objSession1 = Session()


In [64]:
# Creating objects for each Shop to be added in db
shop1 = Shop(shop_name = "PretManger", 
      shop_addressLine1 = "RedLionLane",
      shop_postalCode='EX46RY',
      shop_contact=7826353631,
      shop_cuisine='Beverages Hot,Cold')

shop2 = Shop(shop_name = "Subway", 
      shop_addressLine1 = "QueenSTreet",
      shop_postalCode='EX26RY',
      shop_contact=7826353632,
      shop_cuisine='Sandwiches')

shop3 = Shop(shop_name = "MacD", 
      shop_addressLine1 = "KingSTreet",
      shop_postalCode='EX16RY',
      shop_contact=7826353633,
      shop_cuisine='Burgers')

shop4 = Shop(shop_name = "PizzaHut", 
      shop_addressLine1 = "JackStreet",
      shop_postalCode='EX66RY',
      shop_contact=7826353634,
      shop_cuisine='Pizza')

shop5 = Shop(shop_name = "Karma", 
      shop_addressLine1 = "StJamesPark",
      shop_postalCode='EX46RU',
      shop_contact=7826353635,
      shop_cuisine='IndianChaat')

shop6 = Shop(shop_name = "NamasteExeter", 
      shop_addressLine1 = "Cathedral",
      shop_postalCode='EX46AS',
      shop_contact=7826353636,
      shop_cuisine='TraditionlIndian')

shop7 = Shop(shop_name = "Phoo", 
      shop_addressLine1 = "ChappelSTreet",
      shop_postalCode='EX46AB',
      shop_contact=7826353637,
      shop_cuisine='Chineese')

In [65]:
#Creating objects for each Item to be added in db

item1=Item(item_name="Latte",item_price=2.50)
item2=Item(item_name="Capuchino",item_price=2.50)
item3=Item(item_name="Pinapple Smoothie",item_price=3.00)
item4=Item(item_name="Hot Chocolate",item_price=3.25)
item5=Item(item_name="Chai Latte",item_price=2.75)
item6=Item(item_name="Veg Sandwich",item_price=4)
item7=Item(item_name="Chicken Sandwich",item_price=4)
item8=Item(item_name="Egg Sandwich",item_price=4)
item9=Item(item_name="Pork Sandwich",item_price=4)
item10=Item(item_name="Lamb Sandwich",item_price=4)
item11=Item(item_name="MaC Vegie",item_price=6)
item12=Item(item_name="MaC Aloo Tikki",item_price=6)
item13=Item(item_name="MaC Chicken",item_price=6)
item14=Item(item_name="MaC Puff",item_price=6)
item15=Item(item_name="Chicken Nugets",item_price=6)
item16=Item(item_name="Veg Paradise Pizza",item_price=9)
item17=Item(item_name="Cheese Pizza",item_price=9)
item18=Item(item_name="Chicken Pizza",item_price=9)
item19=Item(item_name="Mexican Pizza",item_price=9)
item20=Item(item_name="Garlick Bread",item_price=5.50)
item21=Item(item_name="Paani Puri",item_price=6)
item22=Item(item_name="Bhel",item_price=10)
item23=Item(item_name="Dahi Puri",item_price=9)
item24=Item(item_name="Pav Kachori",item_price=6.50)
item25=Item(item_name="Aloo Chaat",item_price=4)
item26=Item(item_name="Daal Chawal",item_price=10)
item27=Item(item_name="Chole Bhature",item_price=12)
item28=Item(item_name="North Indian Thaali",item_price=15)
item29=Item(item_name="Maharashtrian Thali",item_price=15)
item30=Item(item_name="Chicken Thaali",item_price=15)
item31=Item(item_name="Veg Fried Rice",item_price=8)
item32=Item(item_name="Chicken Fried Rice",item_price=9)
item33=Item(item_name="Veg Hakka Noodles",item_price=8)
item34=Item(item_name="Burnt Garlic Rice",item_price=9)
item35=Item(item_name="Chicken Noodles",item_price=9)

item36=Item(item_name="CokeCan",item_price=1)
item37=Item(item_name="PepsiCan",item_price=1)
item38=Item(item_name="PepsiMaxCan",item_price=1)
item39=Item(item_name="DietCokeCan",item_price=1)
item40=Item(item_name="FantaCan",item_price=1)


In [66]:
#Adding items to shop by using a collection attribute method append()

################# adding items which are unique in each shop #################
shop1.items.append(item1)
shop1.items.append(item2)
shop1.items.append(item3)
shop1.items.append(item4)
shop1.items.append(item5)

shop2.items.append(item6)
shop2.items.append(item7)
shop2.items.append(item8)
shop2.items.append(item9)
shop2.items.append(item10)

shop3.items.append(item11)
shop3.items.append(item12)
shop3.items.append(item13)
shop3.items.append(item14)
shop3.items.append(item15)

shop4.items.append(item16)
shop4.items.append(item17)
shop4.items.append(item18)
shop4.items.append(item19)
shop4.items.append(item20)

shop5.items.append(item21)
shop5.items.append(item22)
shop5.items.append(item23)
shop5.items.append(item24)
shop5.items.append(item25)

shop6.items.append(item26)
shop6.items.append(item27)
shop6.items.append(item28)
shop6.items.append(item29)
shop6.items.append(item30)

shop7.items.append(item31)
shop7.items.append(item32)
shop7.items.append(item33)
shop7.items.append(item34)
shop7.items.append(item35)

################# adding items which are available in each shop #################

shop1.items.append(item36)
shop1.items.append(item37)
shop1.items.append(item38)
shop1.items.append(item39)
shop1.items.append(item40)

shop2.items.append(item36)
shop2.items.append(item37)
shop2.items.append(item38)
shop2.items.append(item39)
shop2.items.append(item40)

shop3.items.append(item36)
shop3.items.append(item37)
shop3.items.append(item38)
shop3.items.append(item39)
shop3.items.append(item40)

shop4.items.append(item36)
shop4.items.append(item37)
shop4.items.append(item38)
shop4.items.append(item39)
shop4.items.append(item40)

shop5.items.append(item36)
shop5.items.append(item37)
shop5.items.append(item38)
shop5.items.append(item39)
shop5.items.append(item40)

shop6.items.append(item36)
shop6.items.append(item37)
shop6.items.append(item38)
shop6.items.append(item39)
shop6.items.append(item40)

shop7.items.append(item36)
shop7.items.append(item37)
shop7.items.append(item38)
shop7.items.append(item39)
shop7.items.append(item40)

In [67]:
try :
    objSession1.add(shop1)
    objSession1.add(shop2)
    objSession1.add(shop3)
    objSession1.add(shop4)
    objSession1.add(shop5)
    objSession1.add(shop6)
    objSession1.add(shop7)
    objSession1.commit()
    print("\nShops and items added successfully.")

except exc.IntegrityError :
    objSession1.rollback()
    print("\nShops and items already present.")


2022-03-30 19:15:27,169 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:15:27,173 INFO sqlalchemy.engine.Engine INSERT INTO items (item_name, item_price) VALUES (?, ?)
2022-03-30 19:15:27,174 INFO sqlalchemy.engine.Engine [generated in 0.00110s] ('Latte', 2.5)
2022-03-30 19:15:27,175 INFO sqlalchemy.engine.Engine ROLLBACK

Shops and items already present.


In [68]:
#Retreiving items provided by each shop using join over multiple tables

for shopItem in objSession1.query(Shop, Item).filter(Link.shop_id == Shop.id, 
   Link.item_id == Item.id).order_by(Link.shop_id,Link.item_id).all():
    print ("ShopID: {} ShopName: {} ItemID: {} ItemName: {} ItemPrice:{}".format(shopItem.Shop.id,shopItem.Shop.shop_name,shopItem.Item.id,shopItem.Item.item_name,shopItem.Item.item_price))


2022-03-30 19:15:37,069 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:15:37,072 INFO sqlalchemy.engine.Engine SELECT shops.id AS shops_id, shops.shop_name AS shops_shop_name, shops.shop_cuisine AS shops_shop_cuisine, shops."shop_addressLine1" AS "shops_shop_addressLine1", shops."shop_postalCode" AS "shops_shop_postalCode", shops.shop_contact AS shops_shop_contact, items.id AS items_id, items.item_name AS items_item_name, items.item_price AS items_item_price 
FROM shops, items, link 
WHERE link.shop_id = shops.id AND link.item_id = items.id ORDER BY link.shop_id, link.item_id
2022-03-30 19:15:37,073 INFO sqlalchemy.engine.Engine [generated in 0.00094s] ()
ShopID: 1 ShopName: PretManger ItemID: 1 ItemName: Latte ItemPrice:2.500
ShopID: 1 ShopName: PretManger ItemID: 2 ItemName: Capuchino ItemPrice:2.500
ShopID: 1 ShopName: PretManger ItemID: 3 ItemName: Pinapple Smoothie ItemPrice:3.000
ShopID: 1 ShopName: PretManger ItemID: 4 ItemName: Hot Chocolate ItemPrice:3.250
ShopID

C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/2265830926.py:3: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  for shopItem in objSession1.query(Shop, Item).filter(Link.shop_id == Shop.id,


In [69]:
#closing open connections and sessions to avoid db curruption
objSession1.close_all()
conn1.close()

2022-03-30 19:15:45,130 INFO sqlalchemy.engine.Engine ROLLBACK


C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/465161591.py:2: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  objSession1.close_all()


### Registering Coupon Codes and retreiving registered coupons

In [71]:
conn2=sqlite3.connect(pathForSQLConn)
cur=conn2.cursor()
objSession2 = Session()

In [14]:
try:
    objSession2.add_all([
    Coupon(coupon_code='ONLINEFOOD001',coupon_discount=10,coupon_validFrom=datetime.date(2021,3,22),coupon_validTill=datetime.date(2022,3,21)),
    Coupon(coupon_code='ONLINEFOOD002',coupon_discount=10,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,21)),
    Coupon(coupon_code='ONLINEFOOD003',coupon_discount=10,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,21)),
    Coupon(coupon_code='ONLINEFOOD004',coupon_discount=25,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,21)),
    Coupon(coupon_code='ONLINEFOOD005',coupon_discount=25,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,21)),
    Coupon(coupon_code='ONLINEFOOD006',coupon_discount=25,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,21)),
    Coupon(coupon_code='ONLINEFOOD007',coupon_discount=50,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,1)),
    Coupon(coupon_code='ONLINEFOOD008',coupon_discount=50,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,2)),
    Coupon(coupon_code='ONLINEFOOD009',coupon_discount=50,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,2)),
    Coupon(coupon_code='ONLINEFOOD010',coupon_discount=50,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,2)),
    Coupon(coupon_code='ONLINEFOOD011',coupon_discount=50,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,2)),
    Coupon(coupon_code='ONLINEFOOD012',coupon_discount=50,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,2)),
    Coupon(coupon_code='ONLINEFOOD013',coupon_discount=50,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,2)),
    Coupon(coupon_code='ONLINEFOOD014',coupon_discount=50,coupon_validFrom=datetime.date(2022,3,22),coupon_validTill=datetime.date(2022,4,2))
    ])
    objSession2.commit()
    print("\nCoupons added successfully")

except exc.IntegrityError :
    objSession2.rollback()
    print("\nCoupon already exists.")


2022-03-30 17:45:23,248 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 17:45:23,250 INFO sqlalchemy.engine.Engine INSERT INTO coupons (coupon_code, coupon_discount, "coupon_validFrom", "coupon_validTill") VALUES (?, ?, ?, ?)
2022-03-30 17:45:23,251 INFO sqlalchemy.engine.Engine [generated in 0.00102s] (('ONLINEFOOD001', 10, '2021-03-22', '2022-03-21'), ('ONLINEFOOD002', 10, '2022-03-22', '2022-04-21'), ('ONLINEFOOD003', 10, '2022-03-22', '2022-04-21'), ('ONLINEFOOD004', 25, '2022-03-22', '2022-04-21'), ('ONLINEFOOD005', 25, '2022-03-22', '2022-04-21'), ('ONLINEFOOD006', 25, '2022-03-22', '2022-04-21'), ('ONLINEFOOD007', 50, '2022-03-22', '2022-04-01'), ('ONLINEFOOD008', 50, '2022-03-22', '2022-04-02')  ... displaying 10 of 14 total bound parameter sets ...  ('ONLINEFOOD013', 50, '2022-03-22', '2022-04-02'), ('ONLINEFOOD014', 50, '2022-03-22', '2022-04-02'))
2022-03-30 17:45:23,253 INFO sqlalchemy.engine.Engine COMMIT

Coupons added successfully


In [15]:
#Update expired coupon status
def updateCouponStatus(objSession):
    objSession.query(Coupon).filter(datetime.date.today()>Coupon.coupon_validTill).update({Coupon.coupon_status:'EXPIRED'}, synchronize_session = False)
    objSession.commit()
    return print("\nStatus Update Complete")

updateCouponStatus(objSession2)

2022-03-30 17:45:34,454 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 17:45:34,455 INFO sqlalchemy.engine.Engine UPDATE coupons SET coupon_status=? WHERE coupons."coupon_validTill" < ?
2022-03-30 17:45:34,456 INFO sqlalchemy.engine.Engine [generated in 0.00057s] ('EXPIRED', '2022-03-30')
2022-03-30 17:45:34,458 INFO sqlalchemy.engine.Engine COMMIT

Status Update Complete


In [72]:
# Displaying all coupons with their Validity dates
registeredCoupons = objSession2.query(Coupon).all()
for cpn in registeredCoupons:
    print("CouponCode:",cpn.coupon_code, "DiscountOffered:",cpn.coupon_discount, "ValidFrom:",cpn.coupon_validFrom, "ValidTill:",cpn.coupon_validTill, "Status:",cpn.coupon_status)


2022-03-30 19:16:07,814 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:16:07,815 INFO sqlalchemy.engine.Engine SELECT coupons.coupon_code AS coupons_coupon_code, coupons.coupon_discount AS coupons_coupon_discount, coupons.coupon_status AS coupons_coupon_status, coupons."coupon_validFrom" AS "coupons_coupon_validFrom", coupons."coupon_validTill" AS "coupons_coupon_validTill" 
FROM coupons
2022-03-30 19:16:07,815 INFO sqlalchemy.engine.Engine [generated in 0.00047s] ()
CouponCode: ONLINEFOOD001 DiscountOffered: 10 ValidFrom: 2021-03-22 ValidTill: 2022-03-21 Status: EXPIRED
CouponCode: ONLINEFOOD002 DiscountOffered: 10 ValidFrom: 2022-03-22 ValidTill: 2022-04-21 Status: ACTIVE
CouponCode: ONLINEFOOD003 DiscountOffered: 10 ValidFrom: 2022-03-22 ValidTill: 2022-04-21 Status: ACTIVE
CouponCode: ONLINEFOOD004 DiscountOffered: 25 ValidFrom: 2022-03-22 ValidTill: 2022-04-21 Status: ACTIVE
CouponCode: ONLINEFOOD005 DiscountOffered: 25 ValidFrom: 2022-03-22 ValidTill: 2022-04-21 Sta

In [73]:
#closing open connections and sessions to avoid db curruption
objSession2.close_all()
conn2.close()

2022-03-30 19:16:13,583 INFO sqlalchemy.engine.Engine ROLLBACK


C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/1054587198.py:2: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  objSession2.close_all()


### Creating an order - One to many relationship (One customer can give multiple orders  and each having multiple items) and Retreiving orders by joining multiple tables


#### Manual insertion of Customer and Orders using relationship established with the help of backpopulates

In [75]:
#creating cursor for sqlite conneciton and session for alchemy
conn3=sqlite3.connect(pathForSQLConn)
cur=conn3.cursor()
objSession3 = Session()


In [19]:
#Creating orders for a customer using one to many relationship.
#Inserting data into multiple tables using the established relationship

rows = [
   Customer(
      cust_name = "Sai",
      cust_email='sai.chakraborty@exe.in',
      cust_addressLine1='125 Red Lion Lane',
      cust_postalCode='EX46RY',
      cust_contact=7826353601,
      placeOrders = [Order(id=1,item_id=1,item_qty=2,shop_id=1),
                     Order(id=1,item_id=2,item_qty=1,shop_id=1),
                     Order(id=1,item_id=8,item_qty=1,shop_id=1)
               ]
       ),
    Customer(
      cust_name = "Tejas",
      cust_email='tejas.kale@exe.in',
      cust_addressLine1='125 Red Lion Lane',
      cust_postalCode='EX46RY',
      cust_contact=7826353633,
      placeOrders = [Order(id=2,item_id=6,item_qty=2,shop_id=1),
                     Order(id=2,item_id=6,item_qty=2,shop_id=2),
                     Order(id=2,item_id=11,item_qty=2,shop_id=2),
                     Order(id=3,item_id=16,item_qty=2,shop_id=3),
                     Order(id=3,item_id=17,item_qty=2,shop_id=3),
                     Order(id=3,item_id=18,item_qty=2,shop_id=3)
               ]
       ),
    Customer(
      cust_name = "Ayush",
      cust_email='ayush.mishra@exe.in',
      cust_addressLine1='exeter 1',
      cust_postalCode='EX67RY',
      cust_contact=7926353601,
      placeOrders = [Order(id=4,item_id=6,item_qty=1,shop_id=4),
                     Order(id=4,item_id=21,item_qty=2,shop_id=4),
                     Order(id=5,item_id=22,item_qty=1,shop_id=4),
                     Order(id=6,item_id=36,item_qty=2,shop_id=7),
                     Order(id=6,item_id=37,item_qty=2,shop_id=7),
                     Order(id=6,item_id=39,item_qty=3,shop_id=7)
               ]
       ),
    Customer(
      cust_name = "Ravi",
      cust_email='ravi,janu@exe.in',
      cust_addressLine1='st david',
      cust_postalCode='EX75RY',
      cust_contact=7826353833,
      placeOrders = [Order(id=7,item_id=1,item_qty=2,shop_id=1),
                     Order(id=7,item_id=11,item_qty=2,shop_id=2),
                     Order(id=7,item_id=16,item_qty=2,shop_id=3),
                     Order(id=7,item_id=21,item_qty=2,shop_id=4),
                     Order(id=7,item_id=26,item_qty=2,shop_id=5),
                     Order(id=7,item_id=31,item_qty=2,shop_id=6),
                     Order(id=7,item_id=36,item_qty=2,shop_id=7)
               ]
       ),
    Customer(
      cust_name = "Apurva",
      cust_email='apurva.singh@exe.in',
      cust_addressLine1='134 sidwell',
      cust_postalCode='EX90RY',
      cust_contact=7955353601,
      placeOrders = [Order(id=8,item_id=8,item_qty=2,shop_id=1),
                     Order(id=8,item_id=9,item_qty=2,shop_id=1),
                     Order(id=8,item_id=38,item_qty=2,shop_id=7),
                     Order(id=9,item_id=31,item_qty=2,shop_id=6),
                     Order(id=9,item_id=32,item_qty=2,shop_id=6),
                     Order(id=9,item_id=33,item_qty=2,shop_id=6)
               ]
       ),
    Customer(
      cust_name = "Amish",
      cust_email='amish.shah@exe.in',
      cust_addressLine1='st david',
      cust_postalCode='EX01RY',
      cust_contact=7926357551,
      placeOrders = [Order(id=10,item_id=38,item_qty=2,shop_id=7),
                     Order(id=11,item_id=38,item_qty=2,shop_id=7),
                     Order(id=12,item_id=38,item_qty=2,shop_id=7),
                     Order(id=13,item_id=38,item_qty=2,shop_id=7),
                     Order(id=14,item_id=38,item_qty=2,shop_id=7),
                     Order(id=15,item_id=38,item_qty=2,shop_id=7)
               ]
       ),
    Customer(
      cust_name = "Shriya",
      cust_email='shriya.johari@exe.in',
      cust_addressLine1='EX 1',
      cust_postalCode='EX87RY',
      cust_contact=7926358911,
      placeOrders = [Order(id=16,item_id=8,item_qty=2,shop_id=1),
                     Order(id=16,item_id=9,item_qty=2,shop_id=1),
                     Order(id=16,item_id=38,item_qty=2,shop_id=7),
                     Order(id=17,item_id=31,item_qty=2,shop_id=6),
                     Order(id=17,item_id=32,item_qty=2,shop_id=6),
                     Order(id=17,item_id=33,item_qty=2,shop_id=6)
               ]
       ),
    Customer(
      cust_name = "Gulam",
      cust_email='gulam.khan@exe.in',
      cust_addressLine1='The Gandhi',
      cust_postalCode='EX34RY',
      cust_contact=7900358911,
      placeOrders = [Order(id=18,item_id=8,item_qty=2,shop_id=1),
                     Order(id=18,item_id=9,item_qty=2,shop_id=1),
                     Order(id=18,item_id=38,item_qty=2,shop_id=7),
                     Order(id=18,item_id=31,item_qty=2,shop_id=6),
                     Order(id=18,item_id=32,item_qty=2,shop_id=6),
                     Order(id=18,item_id=33,item_qty=2,shop_id=6)
               ]
       ),
    Customer(
      cust_name = "Divyansh",
      cust_email='divyansh.upasani@exe.in',
      cust_addressLine1='125 Red Lion Lane',
      cust_postalCode='EX46RY',
      cust_contact=7826363636,
      placeOrders = [Order(id=19,item_id=8,item_qty=2,shop_id=1),
                     Order(id=19,item_id=9,item_qty=2,shop_id=1),
                     Order(id=19,item_id=38,item_qty=2,shop_id=7),
                     Order(id=20,item_id=31,item_qty=2,shop_id=6),
                     Order(id=20,item_id=32,item_qty=2,shop_id=6),
                     Order(id=20,item_id=33,item_qty=2,shop_id=6)
               ]
       )
]

try :
    objSession3.add_all(rows)
    objSession3.commit()

except exc.IntegrityError:
    objSession3.rollback()
    print("\nCustomer and order ID already exist")

2022-03-30 17:46:10,015 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 17:46:10,016 INFO sqlalchemy.engine.Engine INSERT INTO customers (cust_name, cust_email, "cust_addressLine1", "cust_postalCode", cust_contact) VALUES (?, ?, ?, ?, ?)
2022-03-30 17:46:10,016 INFO sqlalchemy.engine.Engine [generated in 0.00049s] ('Sai', 'sai.chakraborty@exe.in', '125 Red Lion Lane', 'EX46RY', 7826353601)
2022-03-30 17:46:10,019 INFO sqlalchemy.engine.Engine INSERT INTO customers (cust_name, cust_email, "cust_addressLine1", "cust_postalCode", cust_contact) VALUES (?, ?, ?, ?, ?)
2022-03-30 17:46:10,020 INFO sqlalchemy.engine.Engine [cached since 0.004334s ago] ('Tejas', 'tejas.kale@exe.in', '125 Red Lion Lane', 'EX46RY', 7826353633)
2022-03-30 17:46:10,021 INFO sqlalchemy.engine.Engine INSERT INTO customers (cust_name, cust_email, "cust_addressLine1", "cust_postalCode", cust_contact) VALUES (?, ?, ?, ?, ?)
2022-03-30 17:46:10,021 INFO sqlalchemy.engine.Engine [cached since 0.005364s ago] ('A

In [76]:
#Displaying all customers and their respective Orders where one customer can have multiple orders

for c,o,i,s in objSession3.query(Customer,Order,Item,Shop).filter(Customer.id == Order.cust_id,Order.item_id==Item.id,Shop.id==Order.shop_id).all():
    print ("customerID:{} CustomerName: {} OrderID: {}  ItemName: {} ItemQty:{} ItemPrice:{} ShopName:{}".format(c.id,c.cust_name,o.id,i.item_name,o.item_qty,i.item_price,s.shop_name))


2022-03-30 19:16:30,702 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:16:30,705 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.cust_name AS customers_cust_name, customers.cust_email AS customers_cust_email, customers."cust_addressLine1" AS "customers_cust_addressLine1", customers."cust_postalCode" AS "customers_cust_postalCode", customers.cust_contact AS customers_cust_contact, orders.id AS orders_id, orders.cust_id AS orders_cust_id, orders.item_id AS orders_item_id, orders.item_qty AS orders_item_qty, orders.shop_id AS orders_shop_id, items.id AS items_id, items.item_name AS items_item_name, items.item_price AS items_item_price, shops.id AS shops_id, shops.shop_name AS shops_shop_name, shops.shop_cuisine AS shops_shop_cuisine, shops."shop_addressLine1" AS "shops_shop_addressLine1", shops."shop_postalCode" AS "shops_shop_postalCode", shops.shop_contact AS shops_shop_contact 
FROM customers, orders, items, shops 
WHERE customers.id = orders.

In [77]:
#Function to retrieve order details for user provided customer id

def seeOrderDetailsForCustomer(inOrdCustID,objSession):
    filteredOrders=objSession.query(Customer,Order,Item,Shop).filter(Customer.id == Order.cust_id,Order.item_id==Item.id,Shop.id==Order.shop_id,Customer.id==inOrdCustID).all()
    if(len(filteredOrders)==0):
        print("\n Customer Doesnot exist")
    else:
        print("\n \n")
        for c,o,i,s in filteredOrders:
            print ("OrderID: {} CustomerName: {} ItemName: {} ItemQty:{} ItemPrice:{} ShopName:{}".format(o.id,c.cust_name, i.item_name,o.item_qty,i.item_price,s.shop_name))
        print("\n \n")

In [79]:
#retrieve order details for a specific customer
try :
    inputOrdCustID = int(input("Enter Customer Id to see orders:"))
    seeOrderDetailsForCustomer(inputOrdCustID,objSession3)
except ValueError:
    print("\nEnter a valid Integer value")

Enter Customer Id to see orders:60
2022-03-30 19:16:59,808 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.cust_name AS customers_cust_name, customers.cust_email AS customers_cust_email, customers."cust_addressLine1" AS "customers_cust_addressLine1", customers."cust_postalCode" AS "customers_cust_postalCode", customers.cust_contact AS customers_cust_contact, orders.id AS orders_id, orders.cust_id AS orders_cust_id, orders.item_id AS orders_item_id, orders.item_qty AS orders_item_qty, orders.shop_id AS orders_shop_id, items.id AS items_id, items.item_name AS items_item_name, items.item_price AS items_item_price, shops.id AS shops_id, shops.shop_name AS shops_shop_name, shops.shop_cuisine AS shops_shop_cuisine, shops."shop_addressLine1" AS "shops_shop_addressLine1", shops."shop_postalCode" AS "shops_shop_postalCode", shops.shop_contact AS shops_shop_contact 
FROM customers, orders, items, shops 
WHERE customers.id = orders.cust_id AND orders.item_id = items.i

#### User Inputs for customer registration details

In [80]:
custName = input("Enter customer Name:")
custEmail=input("Enter a valid customer Email Address:")
custAddressLine1=input("Enter Address Line:")
custPostalCode=input("Enter postal code of length 6 characters")
custContact=input("Enter contact number 10 digit:")

try:
    objSession3.add(Customer(cust_name=custName,cust_email=custEmail,cust_addressLine1=custAddressLine1,cust_postalCode=custPostalCode,cust_contact=custContact))
    objSession3.commit()
    
    print("\n Registered user details are :")
    registeredUser = objSession3.query(Customer).filter(Customer.cust_email==custEmail).all()
    for cust in registeredUser:
        print("Customer ID:",cust.id,"\nCustomer Name:",cust.cust_name,"\nCustomer Email:",cust.cust_email,"\nCustomer Addressline1:",cust.cust_addressLine1,"\nCustomer Postal Code:",cust.cust_postalCode,"\Customer Contact number:",cust.cust_contact)

except  :
    objSession3.rollback()
    print("\n Error while registering customer details")


Enter customer Name:Dasds
Enter a valid customer Email Address:sgdvfds@jhwvefwe.sdhf
Enter Address Line:jevshf
Enter postal code of length 6 charactersFD45GD
Enter contact number 10 digit:256545415
2022-03-30 19:17:21,786 INFO sqlalchemy.engine.Engine INSERT INTO customers (cust_name, cust_email, "cust_addressLine1", "cust_postalCode", cust_contact) VALUES (?, ?, ?, ?, ?)
2022-03-30 19:17:21,786 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ('Dasds', 'sgdvfds@jhwvefwe.sdhf', 'jevshf', 'FD45GD', '256545415')
2022-03-30 19:17:21,790 INFO sqlalchemy.engine.Engine COMMIT

 Registered user details are :
2022-03-30 19:17:22,219 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:17:22,220 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.cust_name AS customers_cust_name, customers.cust_email AS customers_cust_email, customers."cust_addressLine1" AS "customers_cust_addressLine1", customers."cust_postalCode" AS "customers_cust_postalCode", customers.

#### User inputs for food Order

In [81]:
#Function to cehck if the combinaiton of shopID and itemID is valid or not
def linkCheck(inputShopID,inputItemID,cur):
    qry="select shop_id,item_id from link where shop_id=? and item_id=?"
    cur.execute(qry,(inputShopID,inputItemID))
    try:
        linkCHeck=cur.fetchone()[0]
        return(1)

    except:
        return(0)

In [82]:
#Check if the customerId is registered in the data base or not 
def customerCheck(inputCustId,cur):  
    qry="select id from customers where id=?"
    cur.execute(qry,(inputCustId,))
    try:
        custCheck=cur.fetchone()[0]
        return(1)

    except:
        return(0)



In [83]:
# to get new order ID as it is unique in the table and also the primary key 

def getNewOrderId(cur):
    qry="select max(id) from orders"
    cur.execute(qry)
    newOrderID=cur.fetchone()[0]+1

    return newOrderID


In [85]:
inputCustId=int(input("Enter customer id to order food:"))
inputCustIdStatus=customerCheck(inputCustId,cur)
if(inputCustIdStatus==1):
        inputShopID=int(input("Enter shopID to order from (1 to 7):"))
        inputItemID=int(input("Enter ItemID to order (1 to 40) :"))
        linkCheckStatus=linkCheck(inputShopID,inputItemID,cur)
        newOrderID = getNewOrderId(cur)
        if(linkCheckStatus==1):
            inputItemQty=int(input("Enter Item quantity"))
            objSession3.add(Order(id=newOrderID,cust_id=inputCustId,item_id=inputItemID,item_qty=inputItemQty,shop_id=inputShopID))
            objSession3.commit()

            print("\n orders by customer ID {} are as below".format(inputCustId))
            ordersByCust = objSession3.query(Order).filter(Order.cust_id==inputCustId).all()
            for orderByCust in ordersByCust:
                print ("OrderID: {}  ItemID: {} ItemQty:{} ShopId:{}".format(orderByCust.id,orderByCust.item_id,orderByCust.item_qty,orderByCust.shop_id))
        else:
            print("ShopID - ItemID Combinaiton not present. Please see the menue option again and revisit us.")
else:
    print("Customer ID not present. Try again with correct custmer ID")

Enter customer id to order food:11
Enter shopID to order from (1 to 7):7
Enter ItemID to order (1 to 40) :40
Enter Item quantity40
2022-03-30 19:17:54,351 INFO sqlalchemy.engine.Engine INSERT INTO orders (id, cust_id, item_id, item_qty, shop_id) VALUES (?, ?, ?, ?, ?)
2022-03-30 19:17:54,352 INFO sqlalchemy.engine.Engine [generated in 0.00070s] (22, 11, 40, 40, 7)
2022-03-30 19:17:54,354 INFO sqlalchemy.engine.Engine COMMIT

 orders by customer ID 11 are as below
2022-03-30 19:17:54,413 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:17:54,414 INFO sqlalchemy.engine.Engine SELECT orders.id AS orders_id, orders.cust_id AS orders_cust_id, orders.item_id AS orders_item_id, orders.item_qty AS orders_item_qty, orders.shop_id AS orders_shop_id 
FROM orders 
WHERE orders.cust_id = ?
2022-03-30 19:17:54,415 INFO sqlalchemy.engine.Engine [generated in 0.00099s] (11,)
OrderID: 22  ItemID: 40 ItemQty:40 ShopId:7


In [86]:
objSession3.close_all()
conn3.close()

2022-03-30 19:18:02,080 INFO sqlalchemy.engine.Engine ROLLBACK


C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/853103812.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  objSession3.close_all()


### Generate invoice for a order with/without discount coupon

In [87]:
conn4=sqlite3.connect(pathForSQLConn)
cur=conn4.cursor()
objSession4 = Session()


In [88]:
#Function to validate coupon and check expiry
def couponCheck(inputCoupon,cur):  
    qry="select coupon_status from coupons where coupon_code=?"
    cur.execute(qry,(inputCoupon,))
    try:
        coupon_check=cur.fetchone()[0]
        return(coupon_check)

    except:
        return("INVALID")

In [89]:
#creating function for inserting invoice details into invoice table based on order inputs

def generateInvoiceWithCoupon(orderID,CouponCode,cur,objSession):
    #Connecting to the database
    
    #Query to find total sum of item prices for a order
    qry1 = '''select sum(items.item_price*orders.item_qty) from items 
        join orders on (orders.item_id=items.id)
        where orders.id=?'''
    cur.execute(qry1,(orderID,))
    totalAmount=cur.fetchone()[0]
    
    #Query to discount offered for a coupon_code
    qry2 = '''select coupon_discount from coupons
    where coupon_code=?
    '''
    cur.execute(qry2,(CouponCode,))
    discountOffered=cur.fetchone()[0]    
    #Calculating final invoice amount
    invoiceAmount = decimal.Decimal(totalAmount) - decimal.Decimal(totalAmount*(discountOffered/100))

    
    #inserting data into invoce table based on the data obtained above
    #objSession = Session()
    objSession.add_all([
    Invoice(order_id=orderID,total_amt=totalAmount,discouted_price=invoiceAmount,coupon_code=CouponCode,invoice_date=datetime.date.today())])
    conn4.commit() 
    objSession.commit()
    return 1

In [90]:
#creating function for inserting invoice details into invoice table based on order inputs

def generateInvoiceWithoutCoupon(orderID,cur,objSession):
    #Connecting to the database
    
    #Query to find total sum of item prices for a order
    qry1 = '''select sum(items.item_price*orders.item_qty) from items 
        join orders on (orders.item_id=items.id)
        where orders.id=?'''
    cur.execute(qry1,(orderID,))
    totalAmount=cur.fetchone()[0]
    
    invoiceAmount = totalAmount
    
    #inserting data into invoce table based on the data obtained above
    objSession.add_all([
    Invoice(order_id=orderID,total_amt=totalAmount,discouted_price=invoiceAmount,coupon_code="NA",invoice_date=datetime.date.today())])
   
    conn4.commit()    
    objSession.commit()
    return 1

In [91]:
#OrderID check funciton to check the existance of orderID
def orderIdCheck(orderIdForInvoice,cur):  
    qry="select DISTINCT id from orders where id=?"
    cur.execute(qry,(orderIdForInvoice,))
    try:
        coupon_check=cur.fetchone()[0]
        return(1)

    except:
        return(0)

In [92]:
#Function to check orderID present in invoice table or not

def orderIdInInvoice(orderIdForInvoice,cur):  
    qry="select order_id from invoice where order_id=?"
    cur.execute(qry,(orderIdForInvoice,))
    try:
        coupon_check=cur.fetchone()[0]
        return(1)

    except:
        return(0)

In [93]:
inputOrderIdForInvoice = int(input("Provide orderID to generate invoice for:"))
orderIdCheckStatus=orderIdCheck(inputOrderIdForInvoice,cur)
invoiceCheckForOrderID = orderIdInInvoice(inputOrderIdForInvoice,cur)
if (invoiceCheckForOrderID==0):
    if(orderIdCheckStatus==1):
            print("\nOrder ID found. invoice can be generated....")

            inputCouponChoice=input ('DO you have a coupon to use (Y/N):')
            if (inputCouponChoice=='Y'):
                inputCoupon=input ('Enter Coupon code:')
                couponCheckStatus=couponCheck(inputCoupon,cur)
                if(couponCheckStatus=='ACTIVE' and couponCheckStatus!='USED'):
                    print("\nCoupon is ACTIVE. Discount is getting applied.")
                    generateInvoiceWithCoupon(inputOrderIdForInvoice,inputCoupon,cur,objSession4)
                    #Update Coupon status to used from active
                    objSession4.query(Coupon).filter(Coupon.coupon_code==inputCoupon).update({Coupon.coupon_status:'USED'}, synchronize_session = False)
                    objSession4.commit()
                    print("\n Invoice generation successfull.")
                    print("\n ############################## Generated invoice details ############################## \n")
                    generatedInvoiceForID = objSession4.query(Invoice).filter(Invoice.order_id==inputOrderIdForInvoice).all()
                    for inv in generatedInvoiceForID:
                        print("InvoiceID:",inv.invoice_id,"\nOrderID:",inv.order_id,"\ntotalAmount:",inv.total_amt,"\nCoupon Applied:",inv.coupon_code,"\nDiscountedPrice:",inv.discouted_price, "\nInvoiceDate:",inv.invoice_date)
                elif (couponCheckStatus=='USED'):
                    print("\nCoupon is already used. Invoice can not be generated.")
                elif (couponCheckStatus=='EXPIRED'):
                    print("\nCoupon is EXPIRED. Invoice can not be generayted. Try again with ACTIVE coupon code.")
                else :
                    print("\nCoupon is invalid. Try generating invoice without coupon.")
            else :
                print("\nNo discount can be applied.\n")
                generateInvoiceWithoutCoupon(inputOrderIdForInvoice,cur,objSession4)
                print("\n Invoice generation successfull.")
                print("\n ############################## Generated invoice details ############################## \n")
                for inv in generatedInvoiceForID:
                    print("InvoiceID:",inv.invoice_id,"\nOrderID:",inv.order_id,"\ntotalAmount:",inv.total_amt,"\nCoupon Applied:",inv.coupon_code,"\nDiscountedPrice:",inv.discouted_price, "\nInvoiceDate:",inv.invoice_date)

    else :
            print("OrderID not present. please try again with valid orderID to generate onvoice.")
else:
    print("Invoice Already generated for the orderID. Can not generated the onvoice again.")

#ONLINEFOOD001


Provide orderID to generate invoice for:2

Order ID found. invoice can be generated....
DO you have a coupon to use (Y/N):Y
Enter Coupon code:ONLINEFOOD006

Coupon is ACTIVE. Discount is getting applied.
2022-03-30 19:19:13,667 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:19:13,670 INFO sqlalchemy.engine.Engine INSERT INTO invoice (order_id, total_amt, discouted_price, coupon_code, invoice_date) VALUES (?, ?, ?, ?, ?)
2022-03-30 19:19:13,670 INFO sqlalchemy.engine.Engine [generated in 0.00045s] (2, 12.0, 9.0, 'ONLINEFOOD006', '2022-03-30')
2022-03-30 19:19:13,673 INFO sqlalchemy.engine.Engine COMMIT
2022-03-30 19:19:14,074 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:19:14,075 INFO sqlalchemy.engine.Engine UPDATE coupons SET coupon_status=? WHERE coupons.coupon_code = ?
2022-03-30 19:19:14,076 INFO sqlalchemy.engine.Engine [generated in 0.00066s] ('USED', 'ONLINEFOOD006')
2022-03-30 19:19:14,078 INFO sqlalchemy.engine.Engine COMMIT

 Invoice generation s

C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/3954655447.py:20: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  generatedInvoiceForID = objSession4.query(Invoice).filter(Invoice.order_id==inputOrderIdForInvoice).all()


In [94]:
# Displaying all invoices generated
generatedInvoices = objSession4.query(Invoice).order_by(Invoice.order_id).all()
for inv in generatedInvoices:
        print("OrderID:",inv.order_id,"InvoiceID:",inv.invoice_id,"totalAmount:",inv.total_amt,"Coupon Applied:",inv.coupon_code,"DiscountedPrice:",inv.discouted_price,"InvoiceDate:",inv.invoice_date)


2022-03-30 19:19:20,369 INFO sqlalchemy.engine.Engine SELECT invoice.invoice_id AS invoice_invoice_id, invoice.order_id AS invoice_order_id, invoice.total_amt AS invoice_total_amt, invoice.discouted_price AS invoice_discouted_price, invoice.coupon_code AS invoice_coupon_code, invoice.invoice_date AS invoice_invoice_date 
FROM invoice ORDER BY invoice.order_id
2022-03-30 19:19:20,370 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ()
OrderID: 2 InvoiceID: 1 totalAmount: 12.000 Coupon Applied: ONLINEFOOD006 DiscountedPrice: 9.000 InvoiceDate: 2022-03-30


In [95]:
objSession4.close_all()
conn4.close()

2022-03-30 19:19:24,812 INFO sqlalchemy.engine.Engine ROLLBACK


C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/1356077224.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  objSession4.close_all()


### DELETE RELATED RECORD FROM CHILD TABLE (CASCADE DELETE IN SQLite)

In [96]:
#creating cursor for sqlite conneciton and session for alchemy
conn5=sqlite3.connect(pathForSQLConn)
cur=conn5.cursor()
objSession5 = Session()

In [97]:
#Displaying all customers and their respective Orders where one customer can have multiple orders

for c,o,i,s in objSession5.query(Customer,Order,Item,Shop).filter(Customer.id == Order.cust_id,Order.item_id==Item.id,Shop.id==Order.shop_id).all():
    print ("customerID:{} OrderID: {} CustomerName: {} ItemName: {} ItemQty:{} ItemPrice:{} ShopName:{}".format(c.id,o.id,c.cust_name, i.item_name,o.item_qty,i.item_price,s.shop_name))


2022-03-30 19:19:29,523 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:19:29,524 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.cust_name AS customers_cust_name, customers.cust_email AS customers_cust_email, customers."cust_addressLine1" AS "customers_cust_addressLine1", customers."cust_postalCode" AS "customers_cust_postalCode", customers.cust_contact AS customers_cust_contact, orders.id AS orders_id, orders.cust_id AS orders_cust_id, orders.item_id AS orders_item_id, orders.item_qty AS orders_item_qty, orders.shop_id AS orders_shop_id, items.id AS items_id, items.item_name AS items_item_name, items.item_price AS items_item_price, shops.id AS shops_id, shops.shop_name AS shops_shop_name, shops.shop_cuisine AS shops_shop_cuisine, shops."shop_addressLine1" AS "shops_shop_addressLine1", shops."shop_postalCode" AS "shops_shop_postalCode", shops.shop_contact AS shops_shop_contact 
FROM customers, orders, items, shops 
WHERE customers.id = orders.

In [98]:
custDeleteInput=int(input("\nEnter CustomerID to be deleted:"))
customerCheckStatus=customerCheck(custDeleteInput,cur)
if(customerCheckStatus==1):
    recordSetCustOrdersDelete = objSession5.query(Customer).get(custDeleteInput)
    print("############### CustomerID To be deleted is :{} ###############".format(recordSetCustOrdersDelete.id))
    print ("############### Customer and respective Orders Before Delete ####################")
    objSession5.delete(recordSetCustOrdersDelete)
    objSession5.commit()
    conn5.commit()
    #for c,o,i,s in objSession5.query(Customer,Order,Item,Shop).filter(Customer.id == Order.cust_id,Order.item_id==Item.id,Shop.id==Order.shop_id).all():
     #   print ("OrderID: {} CustomerName: {} ItemName: {} ItemQty:{} ItemPrice:{} ShopName:{}".format(o.id,c.cust_name, i.item_name,o.item_qty,i.item_price,s.shop_name))

    
else:
    print("Customer does not exist.")



Enter CustomerID to be deleted:3
2022-03-30 19:19:36,084 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.cust_name AS customers_cust_name, customers.cust_email AS customers_cust_email, customers."cust_addressLine1" AS "customers_cust_addressLine1", customers."cust_postalCode" AS "customers_cust_postalCode", customers.cust_contact AS customers_cust_contact 
FROM customers 
WHERE customers.id = ?
2022-03-30 19:19:36,085 INFO sqlalchemy.engine.Engine [generated in 0.00108s] (3,)
############### CustomerID To be deleted is :3 ###############
############### Customer and respective Orders Before Delete ####################
2022-03-30 19:19:36,088 INFO sqlalchemy.engine.Engine SELECT orders.id AS orders_id, orders.cust_id AS orders_cust_id, orders.item_id AS orders_item_id, orders.item_qty AS orders_item_qty, orders.shop_id AS orders_shop_id 
FROM orders 
WHERE ? = orders.cust_id
2022-03-30 19:19:36,089 INFO sqlalchemy.engine.Engine [generated in 0.00078s] (3,)


In [99]:
print ("############### List of customers after delete ###############")
for c in objSession5.query(Customer).all():
       print ("Customer ID:{} CustomerName:{} CustomerEmail:{} ".format(c.id,c.cust_name,c.cust_email))
    #objSession5.commit()
print ("############### List of orders after delete ###############")
for o in objSession5.query(Order).all():
    print ("OrderID:{} OrderedItemID:{} ItemQty:{} OrderedFromShopID:{} ".format(o.id,o.item_id,o.item_qty,o.shop_id))
    #objSession5.commit()
print ("############### End Of Result ####################")


############### List of customers after delete ###############
2022-03-30 19:19:39,494 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:19:39,495 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.cust_name AS customers_cust_name, customers.cust_email AS customers_cust_email, customers."cust_addressLine1" AS "customers_cust_addressLine1", customers."cust_postalCode" AS "customers_cust_postalCode", customers.cust_contact AS customers_cust_contact 
FROM customers
2022-03-30 19:19:39,496 INFO sqlalchemy.engine.Engine [generated in 0.00097s] ()
Customer ID:1 CustomerName:Sai CustomerEmail:sai.chakraborty@exe.in 
Customer ID:2 CustomerName:Tejas CustomerEmail:tejas.kale@exe.in 
Customer ID:4 CustomerName:Ravi CustomerEmail:ravi,janu@exe.in 
Customer ID:5 CustomerName:Apurva CustomerEmail:apurva.singh@exe.in 
Customer ID:6 CustomerName:Amish CustomerEmail:amish.shah@exe.in 
Customer ID:7 CustomerName:Shriya CustomerEmail:shriya.johari@exe.in 
Customer ID

In [100]:
objSession5.close_all()
conn5.close()

2022-03-30 19:19:47,632 INFO sqlalchemy.engine.Engine ROLLBACK


C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/2516157885.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  objSession5.close_all()


### Update price of Items for a ShopID

In [101]:
conn6=sqlite3.connect(pathForSQLConn)
cur=conn6.cursor()
objSession6 = Session()

In [102]:
def updateItemPrice(cur,shopID,percentIncrease):
    
    #Query to find total sum of item prices for a order
    qry1 = '''update items
    set item_price=item_price+((item_price*?)/100)
    from link as l
    where l.shop_id=? and l.item_id=items.id
    '''
    cur.execute(qry1,(percentIncrease,shopID,))
    conn6.commit()
    return 1

In [103]:
def printAllShopsAndItems(sessionObj):
    for shopItem in sessionObj.query(Shop, Item).filter(Link.shop_id == Shop.id, Link.item_id == Item.id).order_by(Link.shop_id,Link.item_id).all():
        print ("ShopID: {} ShopName: {} ItemID: {} ItemName: {} ItemPrice:{}".format(shopItem.Shop.id,shopItem.Shop.shop_name,shopItem.Item.id,shopItem.Item.item_name,shopItem.Item.item_price))    


In [104]:

inShopId=int(input ('Enter shopID to update item price'))
inPriceIncrease=int(input ('Enter % to increate item price'))

print(" ##################### All Items before update ##################### ")
printAllShopsAndItems(objSession6)
updateItemPrice(cur,inShopId,inPriceIncrease)
conn6.commit()
objSession6.commit()
print(" ##################### All Items after update ##################### ")
printAllShopsAndItems(Session())



Enter shopID to update item price7
Enter % to increate item price15
 ##################### All Items before update ##################### 
2022-03-30 19:20:02,228 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:20:02,229 INFO sqlalchemy.engine.Engine SELECT shops.id AS shops_id, shops.shop_name AS shops_shop_name, shops.shop_cuisine AS shops_shop_cuisine, shops."shop_addressLine1" AS "shops_shop_addressLine1", shops."shop_postalCode" AS "shops_shop_postalCode", shops.shop_contact AS shops_shop_contact, items.id AS items_id, items.item_name AS items_item_name, items.item_price AS items_item_price 
FROM shops, items, link 
WHERE link.shop_id = shops.id AND link.item_id = items.id ORDER BY link.shop_id, link.item_id
2022-03-30 19:20:02,230 INFO sqlalchemy.engine.Engine [cached since 265.2s ago] ()
ShopID: 1 ShopName: PretManger ItemID: 1 ItemName: Latte ItemPrice:2.500
ShopID: 1 ShopName: PretManger ItemID: 2 ItemName: Capuchino ItemPrice:2.500
ShopID: 1 ShopName: PretManger I

In [105]:
objSession6.close_all()
conn6.close()

2022-03-30 19:20:16,870 INFO sqlalchemy.engine.Engine ROLLBACK


C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/1140775.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  objSession6.close_all()


### Provide feedback for a order

In [106]:
#Connection for the sqlite DB
conn7=sqlite3.connect(pathForSQLConn)
cur=conn7.cursor()
objSession7 = Session()

In [107]:
#Checking whether provided order ID is present in the table ot not

def checkOrderId(inputOrderId,cur):  
    qry="select id from orders where id=?;"
    cur.execute(qry,(inputOrderId,))
    try:
        coupon_check=cur.fetchone()[0]
        #print(coupon_check)
        return(True)

    except:
        return(False)

In [108]:
#Checking whether Feedback for an OrderID is already present or not.
# As it is 1 to 1 relationship, one Order ID can have only One feedback

def checkOrderInFeedbackTable(inputOrderId,cur):  
    qry="select feedback_id from feedback where feedback_id=?;"
    cur.execute(qry,(inputOrderId,))
    try:
        OrderIdInFeedbackCheck=cur.fetchone()[0]
        #print(OrderIdInFeedbackCheck)
        return(True)

    except:
        return(False)

In [111]:

OrderIDForFeedback = int(input("Enter OrderID to provide feedback:"))
OrderCheckStatus=checkOrderId(OrderIDForFeedback,cur)
#print(OrderCheckStatus)
OrderIdInFeedbackCheck=checkOrderInFeedbackTable(OrderIDForFeedback,cur)
#print(OrderIdInFeedbackCheck)

if(OrderCheckStatus==True):
    if(OrderIdInFeedbackCheck==False):
        OrderFeedbackInput=input("Please provide Feedback for the selected orderID")
        objSession7.add(Feedback(order_id=OrderIDForFeedback,feedback_text=OrderFeedbackInput))
        objSession7.commit()
    else:
        print("\nFeedback for this order is already present.")  
else:
     print("\nInvalid OrderID. Please Try again with correct OrderID")



Enter OrderID to provide feedback:1

Feedback for this order is already present.


In [112]:
#Retreiving Order Feedback
providedFeedbacks = objSession7.query(Feedback).order_by(Feedback.feedback_id).all()
    # Displaying all coupons with their Validity dates
for fdbk in providedFeedbacks:
        print("FeedbackID:",fdbk.feedback_id,"OrderID:",fdbk.order_id,"Provided Feedback:",fdbk.feedback_text)


2022-03-30 19:20:42,546 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-30 19:20:42,548 INFO sqlalchemy.engine.Engine SELECT feedback.feedback_id AS feedback_feedback_id, feedback.order_id AS feedback_order_id, feedback.feedback_text AS feedback_feedback_text 
FROM feedback ORDER BY feedback.feedback_id
2022-03-30 19:20:42,549 INFO sqlalchemy.engine.Engine [generated in 0.00101s] ()
FeedbackID: 1 OrderID: 1 Provided Feedback: Gooddd


In [113]:
conn7.close()
objSession7.close_all()

2022-03-30 19:20:44,732 INFO sqlalchemy.engine.Engine ROLLBACK


C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/2877651611.py:2: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  objSession7.close_all()


# Non-Relational Database implementation with Redis

### Importing Redis and Selecting the database in which need to enter the data 

In [114]:
#Importing redis for python
import redis

#sessing connection with port number nad database number
r2 =redis.Redis(host='localhost', port=6379, db=6)

#printing connection status
print(r2.ping())

True


### Inserting the data  

In [115]:
customer1 = "c1"
r2.hmset(customer1,  {1:"latte", 2:"capucchino",8: "Egg Sandwich",15: "Chicken Nugets"})

customer2 = "c2"
r2.hmset(customer2,  {1:"latte", 2:"capucchino",8: "Egg Sandwich"})

customer3 = "c3"
r2.hmset(customer3,  {1:"latte", 2:"capucchino"})

customer4 = "c4"
r2.hmset(customer4,  {1:"latte", 2:"capucchino",13: "MaC Chicken",10: "Lamb Sandwich"})

customer5 = "c5"
r2.hmset(customer5,  {1:"latte", 2:"capucchino",8: "Egg Sandwich",10: "Lamb Sandwich"})

customer6 = "c6"
r2.hmset(customer6,  {1:"latte", 2:"capucchino"})

customer7 = "c7"
r2.hmset(customer7,  {1:"latte", 2:"capucchino",13: "MaC Chicken"})

C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/3554284889.py:2: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r2.hmset(customer1,  {1:"latte", 2:"capucchino",8: "Egg Sandwich",15: "Chicken Nugets"})
C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/3554284889.py:5: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r2.hmset(customer2,  {1:"latte", 2:"capucchino",8: "Egg Sandwich"})
C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/3554284889.py:8: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r2.hmset(customer3,  {1:"latte", 2:"capucchino"})
C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/3554284889.py:11: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r2.hmset(customer4,  {1:"latte", 2:"capucchino",13: "MaC Chicken",10: "Lamb Sandwich"})
C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/3554284889.py:14: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.h

True

### Retreiving stored data by checking CustID from customers table

In [116]:
print(r2.hgetall("c1"))

{b'1': b'latte', b'2': b'capucchino', b'8': b'Egg Sandwich', b'15': b'Chicken Nugets'}


### Applying Time to leave concept. Expiring an item in database  after 60 sec. 

In [117]:
print(r2.keys("c1*"))
r2.expire(name=customer1, time=10)

[b'c1']


True

In [124]:
print(r2.ttl(customer1))
if(r2.ttl(customer1)<0):
        print("item out of stock")

-2
item out of stock


# Operation interdependant on SQLite and Redis

In [125]:
conn8=sqlite3.connect(pathForSQLConn)
cur=conn8.cursor()
objSession8 = Session()

In [126]:
#Creating wishlists 

customer1 = "1"
r2.hmset(customer1,  {1:"latte", 2:"capucchino",8: "Egg Sandwich",15: "Chicken Nugets"})

customer2 = "2"
r2.hmset(customer2,  {1:"latte", 2:"capucchino",8: "Egg Sandwich"})

customer3 = "3"
r2.hmset(customer3,  {1:"latte", 2:"capucchino"})

customer4 = "4"
r2.hmset(customer4,  {1:"latte", 2:"capucchino",13: "MaC Chicken",10: "Lamb Sandwich"})

customer5 = "5"
r2.hmset(customer5,  {1:"latte", 2:"capucchino",8: "Egg Sandwich",10: "Lamb Sandwich"})

customer6 = "6"
r2.hmset(customer6,  {1:"latte", 2:"capucchino"})

customer7 = "7"
r2.hmset(customer7,  {1:"latte", 2:"capucchino",13: "MaC Chicken"})

C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/1775593190.py:4: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r2.hmset(customer1,  {1:"latte", 2:"capucchino",8: "Egg Sandwich",15: "Chicken Nugets"})
C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/1775593190.py:7: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r2.hmset(customer2,  {1:"latte", 2:"capucchino",8: "Egg Sandwich"})
C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/1775593190.py:10: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r2.hmset(customer3,  {1:"latte", 2:"capucchino"})
C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/1775593190.py:13: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r2.hmset(customer4,  {1:"latte", 2:"capucchino",13: "MaC Chicken",10: "Lamb Sandwich"})
C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/1775593190.py:16: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.

True

In [127]:
#Setting Expiry for some available wishlists

r2.expire(name=customer1,time=60)
r2.expire(name=customer2,time=120)
r2.expire(name=customer3,time=30)
r2.expire(name=customer4,time=5)
r2.expire(name=customer5,time=10)
r2.expire(name=customer6,time=300)
r2.expire(name=customer7,time=20)

True

In [133]:
custIDinputForWishlist=int(input("Enter CustID to retrieve wishlist:"))
customerCheckStatusWishlist=customerCheck(custIDinputForWishlist,cur)
if(customerCheckStatusWishlist==1):
    if(custIDinputForWishlist>=1 and custIDinputForWishlist<=7):
        if(custIDinputForWishlist==1 and (r2.ttl(customer1)>0)):
            print(r2.hgetall(1))
            print("Time To live remaining is:{}".format(r2.ttl(customer1)))    
        elif(custIDinputForWishlist==2 and (r2.ttl(customer2)>0)):
            print(r2.hgetall(1))
            print("Time To live remaining is:{}".format(r2.ttl(customer2)))
        elif(custIDinputForWishlist==3 and (r2.ttl(customer3)>0)):
            print(r2.hgetall(1))
            print("Time To live remaining is:{}".format(r2.ttl(customer3)))
        elif(custIDinputForWishlist==4 and (r2.ttl(customer4)>0)):
            print(r2.hgetall(1))
            print("Time To live remaining is:{}".format(r2.ttl(customer4)))
        elif(custIDinputForWishlist==5 and (r2.ttl(customer5)>0)):
            print(r2.hgetall(1))
            print("Time To live remaining is:{}".format(r2.ttl(customer5)))
        elif(custIDinputForWishlist==6 and (r2.ttl(customer6)>0)):
            print(r2.hgetall(1))
            print("Time To live remaining is:{}".format(r2.ttl(customer6)))
        elif(custIDinputForWishlist==7 and (r2.ttl(customer7)>0)):
            print(r2.hgetall(1))
            print("Time To live remaining is:{}".format(r2.ttl(customer7)))
        else:
            print("Wishlist is Expired. Ask admin to create a wishlist for you")
    else:
        print("Customer Does not have wishlist")
else:
    print("Customer does not exists")

Enter CustID to retrieve wishlist:2
{b'1': b'latte', b'2': b'capucchino', b'8': b'Egg Sandwich', b'15': b'Chicken Nugets'}
Time To live remaining is:93


In [134]:
conn8.close()
objSession8.close_all()

C:\Users\tejas\AppData\Local\Temp/ipykernel_16736/1701735844.py:2: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  objSession8.close_all()
